<a href="https://colab.research.google.com/github/yandexdataschool/MLatMISiS2018/blob/master/02_lab/02-LinearModels_LHCb_PID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample management

In [0]:
!wget https://github.com/hse-aml/hadron-collider-machine-learning/releases/download/Week_2/training.csv.gz

In [0]:
!gunzip training.csv.gz

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
data = pd.read_csv('training.csv')

In [0]:
len(data)

## Feature description
Here, Spd stands for Scintillating Pad Detector, Prs - Preshower, Ecal - electromagnetic calorimeter, Hcal - hadronic calorimeter, Brem denotes traces of the particles that were deflected by detector

Features:

* ID - id value for tracks (presents only in the test file for the submitting purposes)
* Label - string valued observable denoting particle types. Can take values "Electron", "Muon", "Kaon", "Proton", "Pion" and "Ghost". This column is absent in the test file.
* FlagSpd - flag (0 or 1), if reconstructed track passes through Spd
* FlagPrs - flag (0 or 1), if reconstructed track passes through Prs
* FlagBrem - flag (0 or 1), if reconstructed track passes through Brem
* FlagEcal - flag (0 or 1), if reconstructed track passes through Ecal
* FlagHcal - flag (0 or 1), if reconstructed track passes through Hcal
* FlagRICH1 - flag (0 or 1), if reconstructed track passes through the first RICH detector
* FlagRICH2 - flag (0 or 1), if reconstructed track passes through the second RICH detector
* FlagMuon - flag (0 or 1), if reconstructed track passes through muon stations (Muon)
* SpdE - energy deposit associated to the track in the Spd
* PrsE - energy deposit associated to the track in the Prs
* EcalE - energy deposit associated to the track in the Hcal
* HcalE - energy deposit associated to the track in the Hcal
* PrsDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Prs
* BremDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Brem
* TrackP - particle momentum
* TrackPt - particle transverse momentum
* TrackNDoFSubdetector1 - number of degrees of freedom for track fit using hits in the tracking sub-detector1
* TrackQualitySubdetector1 - chi2 quality of the track fit using hits in the tracking sub-detector1
* TrackNDoFSubdetector2 - number of degrees of freedom for track fit using hits in the tracking sub-detector2
* TrackQualitySubdetector2 - chi2 quality of the track fit using hits in the tracking sub-detector2
* TrackNDoF - number of degrees of freedom for track fit using hits in all tracking sub-detectors
* TrackQualityPerNDoF - chi2 quality of the track fit per degree of freedom
* TrackDistanceToZ - distance between track and z-axis (beam axis)
* Calo2dFitQuality - quality of the 2d fit of the clusters in the calorimeter
* Calo3dFitQuality - quality of the 3d fit in the calorimeter with assumption that particle was electron
* EcalDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Ecal
* EcalDLLbeMuon - delta log-likelihood for a particle candidate to be muon using information from Ecal
* EcalShowerLongitudinalParameter - longitudinal parameter of Ecal shower
* HcalDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Hcal
* HcalDLLbeMuon - delta log-likelihood for a particle candidate to be using information from Hcal
* RICHpFlagElectron - flag (0 or 1) if momentum is greater than threshold for electrons to produce Cherenkov light
* RICHpFlagProton - flag (0 or 1) if momentum is greater than threshold for protons to produce Cherenkov light
* RICHpFlagPion - flag (0 or 1) if momentum is greater than threshold for pions to produce Cherenkov light
* RICHpFlagKaon - flag (0 or 1) if momentum is greater than threshold for kaons to produce Cherenkov light
* RICHpFlagMuon - flag (0 or 1) if momentum is greater than threshold for muons to produce Cherenkov light
* RICH_DLLbeBCK - delta log-likelihood for a particle candidate to be background using information from RICH
* RICH_DLLbeKaon - delta log-likelihood for a particle candidate to be kaon using information from RICH
* RICH_DLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from RICH
* RICH_DLLbeMuon - delta log-likelihood for a particle candidate to be muon using information from RICH
* RICH_DLLbeProton - delta log-likelihood for a particle candidate to be proton using information from RICH
* MuonFlag - muon flag (is this track muon) which is determined from muon stations
* MuonLooseFlag muon flag (is this track muon) which is determined from muon stations using looser criteria
* MuonLLbeBCK - log-likelihood for a particle candidate to be not muon using information from muon stations
* MuonLLbeMuon - log-likelihood for a particle candidate to be muon using information from muon stations
* DLLelectron - delta log-likelihood for a particle candidate to be electron using information from all subdetectors
* DLLmuon - delta log-likelihood for a particle candidate to be muon using information from all subdetectors
* DLLkaon - delta log-likelihood for a particle candidate to be kaon using information from all subdetectors
* DLLproton - delta log-likelihood for a particle candidate to be proton using information from all subdetectors
* GhostProbability - probability for a particle candidate to be ghost track. This variable is an output of classification model used in the tracking algorithm.

Delta log-likelihood in the features descriptions means the difference between log-likelihood for the mass hypothesis that a given track is left by some particle (for example, electron) and log-likelihood for the mass hypothesis that a given track is left by a pion (so, DLLpion = 0 and thus we don't have these columns). This is done since most tracks (~80%) are left by pions and in practice we actually need to discriminate other particles from pions. In other words, the null hypothesis is that particle is a pion.


# # Feature enigineering
Feature selection and preprocessing, model validation

In [0]:
data.columns

Let's consider PID between two particle types for simplicity:

In [0]:
data = data[(data.Label == 'Kaon') | (data.Label == 'Pion')].copy()

features = [col for col in data.columns if col != 'Label']
data['Label'] = (data.Label == 'Kaon').astype(float)

print(len(data))

In [0]:
from sklearn import linear_model, metrics, model_selection, preprocessing
train, test = model_selection.train_test_split(data, test_size=0.25)

Selecting the best features is quite an important and non-trivial part of building machine learning models. Scikit-learn has a number of ways to automate this process - to be used with caution - see [this page](https://scikit-learn.org/stable/modules/feature_selection.html) for more details.

At this point we are not going to use these tools, but rather do a really simple thing: will score each feature with `roc_auc_score` to find those giving maximum separation between classes.

In [0]:
# Build an array of scores of form [(feature1, score1), (feature2, score2), ...]
scores = [(f, metrics.roc_auc_score(data.Label, data[f])) for f in features]

# Sort this array by the scores in descending order.
# As AUC is symmetric with respect to 0.5, we'll sort
# by max(score, 1-score):
scores = (sorted(scores, key=lambda x: -max(x[1], 1-x[1])))

In [0]:
# Print top 10:
print("Feature : roc_auc_score \n")
for f, score in scores[:10]:
  print("{} : {}".format(f, score))

So, just a single `DLLkaon` feature gives us an AUC of 94%!
Let's see if we can beat this score.

The simplest thing we can do is to take, say, 10 best features and feed them into a logistic regression model:

In [0]:
top10_features = list(list(zip(*scores))[0][:10])

def get_features(dataset):
  return dataset[top10_features]

model = linear_model.LogisticRegression()

model.fit(get_features(train), train.Label)

preds_train = model.predict_proba(get_features(train))[:,1]
preds_test  = model.predict_proba(get_features(test ))[:,1]

print(metrics.roc_auc_score(train.Label, preds_train))
print(metrics.roc_auc_score(test .Label, preds_test ))

Hmm, that just decreased the score.

Let's look at the range of these features:

In [0]:
for f in top10_features:
  print("{:20s} : ({:10.2f}, {:10.2f})".format(f, data[f].min(), data[f].max()))

We can notice two things:
1.   ranges are very different
2.   some variables have 'unnatural' minimum of -999

Let's discuss problem 1 first. Our model treats its inputs as vectors of $R^M$ space ($M$ is the number of features), and calculates things like dot-product ${\bf W}\cdot{\bf x}$. This assumes that all the components of these vectors are objects of the same nature and have the same units. Obviously this is not the case. We can however emulate this by scaling the components of these vectors to have same variance and mean:



In [0]:
def get_features(dataset):
  return dataset[top10_features]

scaler = preprocessing.RobustScaler()
scaler.fit(get_features(train))

model = linear_model.LogisticRegression()

model.fit(scaler.transform(get_features(train)), train.Label)

preds_train = model.predict_proba(scaler.transform(get_features(train)))[:,1]
preds_test  = model.predict_proba(scaler.transform(get_features(test )))[:,1]

print(metrics.roc_auc_score(train.Label, preds_train))
print(metrics.roc_auc_score(test .Label, preds_test ))

This increased the score slightly.

Now, problem 2. Let's have a look at one of these features with -999 minimum:

In [0]:
plt.hist(data.RICH_DLLbeKaon, bins=300);

Note the standalone peak near -1000 (actually, -999). Looks like some discreet value was used to denote the cases when `RICH_DLLbeKaon` could not be calculated.

The simplest thing we can do with it is to just replace -999 by the mean of the feature, but since in such a way we'll lose this information, let's encode it into a new feature:

In [0]:
def convert_outlier(column, value=-999):
  """
  This function takes a single pandas column and returns a dataframe
  with two columns: same column with all occurrences of `value`
  replaced by mean and a binary `column == value` column
  """
  is_out = (column == value)
  is_out.name += '_out'
  
  mean = column[~is_out].mean()
  column = column.copy()
  column[is_out] = mean
  
  return pd.concat([column, is_out.astype(float)], axis=1)

In [0]:
outlier_columns = [f for f in top10_features if (data[f] == -999).sum() > 0]
print(outlier_columns)

In [0]:
def get_features(dataset):
  return pd.concat([convert_outlier(dataset[f]) if f in outlier_columns else
                    dataset[f] for f in top10_features], axis=1)


Let's go to our final training.

In [0]:
scaler = preprocessing.RobustScaler()
scaler.fit(get_features(train))

model = linear_model.LogisticRegression()

model.fit(scaler.transform(get_features(train)), train.Label)

preds_train = model.predict_proba(scaler.transform(get_features(train)))[:,1]
preds_test  = model.predict_proba(scaler.transform(get_features(test )))[:,1]

print(metrics.roc_auc_score(train.Label, preds_train))
print(metrics.roc_auc_score(test .Label, preds_test ))

In [0]:
fpr_, tpr_, _ = metrics.roc_curve(test.Label, preds_test )
fpr_dll, tpr_dll, _ = metrics.roc_curve(test.Label, test.DLLkaon)

plt.plot(fpr_dll, tpr_dll, label='DLLkaon')
plt.plot(fpr_, tpr_, label='Our Model')
plt.legend()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')

plt.show();

Huh! We've finally beaten the `DLLkaon` score.

But there is no panacea for feature selection/dimensionality reduction. In many real examples you can just choose appropriate feature selection algorithm from sklearn. However, in certain cases you obliged to make feature engineering to accept the compromise between performance measure and expensiveness.

# Resampling methods
## k-fold cross validation
Now let's use the k-fold cross validation technique to ensure this is indeed the case.

k-fold cross-validation randomly divides the data into k blocks of roughly equal size. Each of the blocks is left out in turn and the other k-1 blocks are used to train the model. The held out block is predicted and these predictions are summarized into some type of performance measure (e.g. accuracy, root mean squared error (RMSE), etc.). The k estimates of performance are averaged to get the overall resampled estimate.

In [0]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=1234)
aucs_single = []
aucs_model = []

for i_train, i_test in kf.split(data):
  train = data.iloc[i_train]
  test  = data.iloc[i_test ]
  
  scaler = preprocessing.RobustScaler()
  scaler.fit(get_features(train))
  
  model = linear_model.LogisticRegression()
  model.fit(scaler.transform(get_features(train)), train.Label)

  preds_test = model.predict_proba(
      scaler.transform(get_features(test))
  )[:,1]
  
  aucs_model .append(metrics.roc_auc_score(test.Label, preds_test))
  aucs_single.append(metrics.roc_auc_score(test.Label, test.DLLkaon))

In [0]:
plt.figure(figsize=(7, 1.7))
plt.scatter(aucs_model , [0] * len(aucs_model ), s=100, alpha=0.5, c='r', label='Our model');
plt.scatter(aucs_single, [0] * len(aucs_single), s=100, alpha=0.5, c='b', label='just DLLkaon');
plt.yticks([]);
plt.ylim(-0.2, 0.4)
plt.xlabel("AUC");
plt.legend();